<a href="https://colab.research.google.com/github/Leksandretta/inno_arch/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pytelegrambotapi
!pip3 install ipynb
!pip3 install import-ipynb

import import_ipynb
import time
import pandas as pd
import telebot
from google.colab import drive

from sklearn.linear_model import LogisticRegression
import csv

In [3]:
%cd /content/drive/MyDrive/Colab

/content/drive/MyDrive/Colab


In [4]:
def company(company_data):
    train = pd.read_csv('/content/dataTRAIN.csv')
    test = pd.read_csv('/content/dataTEST.csv')

    with open('/content/dataTEST.csv', 'a') as fp:
        wr = csv.writer(fp, dialect='excel')
        wr.writerow(company_data)

    train.drop(['id','name', 'INN'], axis = 1, inplace = True)
    X_train = train.drop('status', axis = 1)
    y_train = train['status']
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)

    test = pd.read_csv('/content/dataTEST.csv')
    test2 = test['id'].copy()
    test.drop(['id','name', 'INN'], axis = 1, inplace = True)
    y_pred_test = model.predict(test)
    result = pd.DataFrame({'id': test2, 'status': y_pred_test})
    result.to_csv('/content/result.csv', sep=",", index = False)

    return (result[len(result)-1:]['status'])
    print(result)

In [ ]:
bot = telebot.TeleBot('7900613999:AAGvrmLZiChJKNLEM5Ki4itizt7ZlMNj73U')

@bot.message_handler(commands=['start'])
def welcome(message):
    bot.send_message(message.chat.id, 'Для получения информации о возможности заключения '
                                      'государственного контракта укажите: Количество ранее заключенных договоров, '
                                      'Сколько договоров из них исполнено, Сколько договоров из них не исполнено,'
                                      'Является ли компания субъектом МСП (1-да, 0-нет), '
                                      'Является ли компания государственной (1-да, 0-нет)')

@bot.message_handler(content_types=['text'])
def answer(message):
    bot.send_message(message.chat.id, 'Анализируем...')
    user_data = message.text.split()
    user_data.insert(0, 0)
    user_data.insert(4, 'N')
    user_data.insert(5, '7718556679')

    answer = company(user_data)
    if int(answer) == 1:
        bot.send_message(message.chat.id, 'Поставщик надежный. Рекомендация: заключать контракт.')
    elif int(answer) == 0:
        bot.send_message(message.chat.id, 'Поставщик ненадежный. Рекомендация: не заключать контракт.')

    do_again(message)

def do_again(message):
    bot.send_message(message.chat.id, 'Проверить кого-нибудь еще?')

if __name__ == '__main__':
    while True:
        try:
            bot.polling(none_stop=True)
        except ():
            time.sleep(5)

<ipython-input-5-325353f8ed10>:20: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if int(answer) == 1:
<ipython-input-5-325353f8ed10>:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  elif int(answer) == 0:
